<a href="https://colab.research.google.com/github/toitoi11/Naver-Blog-Scraper-with-Selenium/blob/main/crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!pip install webdriver_manager

In [ ]:
!pip install chromedriver-autoinstaller
!pip install --upgrade webdriver_manager

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [ ]:
import urllib.request
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

# Set up the WebDriver
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)

# Input information
client_id = "your client ID"  # Enter your issued client ID
client_secret = "your client secret"  # Enter your issued client secret
quote = input("Please enter a search term: ")  # Get the search term
encText = urllib.parse.quote(quote)
display_num = input("Please specify the number of search results to display (max 100, numbers only): ")  # Get the number of results to display
start_num = input("Please specify the starting point: ")
url = "https://openapi.naver.com/v1/search/blog?query=" + encText + "&display=" + display_num + "&start=" + start_num  # JSON result
# url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # XML result
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id", client_id)
request.add_header("X-Naver-Client-Secret", client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()

if rescode == 200:
    response_body = response.read()
else:
    print("Error Code:" + str(rescode))

body = response_body.decode('utf-8')
print(body)

# Split the body content
list1 = body.split('\n\t\t{\n\t\t\t')
# Extract only Naver blog posts
list1 = [i for i in list1 if 'naver' in i]
print(list1)

# Extract blog titles and links
import re
titles = []
links = []
for i in list1:
    title = re.findall('"title":"(.*?)",\n\t\t\t"link"', i)
    link = re.findall('"link":"(.*?)",\n\t\t\t"description"', i)
    titles.append(title)
    links.append(link)

titles = [r for i in titles for r in i]
links = [r for i in links for r in i]

print('<<Title Collection>>')
print(titles)
print('Total number of titles: ', len(titles), 'items')  # Check the number of titles
print('\n<<Link Collection>>')
print(links)
print('Total number of links: ', len(links), 'items')  # Check the number of links

# Clean up the links (remove unnecessary parts)
blog_links = []
for i in links:
    a = i.replace('\\', '')
    b = a.replace('?Redirect=Log&logNo=', '/')
    blog_links.append(b)

print(blog_links)
print('Number of generated links: ', len(blog_links), 'items')

# Scraping blog contents
import time
from selenium import webdriver

# Install Chrome driver
driver = webdriver.Chrome(options=chrome_options)
driver.implicitly_wait(3)

# Load each blog link and scrape content
contents = []
for i in blog_links:
    # Load each blog link
    driver.get(i)
    time.sleep(1)
    # Access the iframe where the blog post content is located
    driver.switch_to.frame("mainFrame")
    # Scrape the blog post content
    try:
        a = driver.find_element(By.CSS_SELECTOR, 'div.se-main-container').text
        contents.append(a)
    # Handle NoSuchElementException (applies to older versions of the blog)
    except NoSuchElementException:
        a = driver.find_element(By.CSS_SELECTOR, 'div#content-area').text
        contents.append(a)

driver.quit()  # Close the browser
print("<<Content scraping completed.>>")

# Save titles and content to a txt file
total_contents = titles + contents

with open("blog_text.txt", 'w', encoding='utf-8') as text:
    for i in total_contents:
        text.write(i)

# Create a DataFrame for titles, blog links, and content
import pandas as pd

df = pd.DataFrame({'Title': titles, 'Link': blog_links, 'Content': contents})
print(df)

# Save the DataFrame
df.to_csv('{}_blogs.csv'.format(quote), encoding='utf-8-sig', index=False)


검색어를 입력해주세요.: 시 모음
검색 출력결과 갯수를 적어주세요.(최대100, 숫자만 입력): 100
시작 지점을 설정해주세요.: 300
{
	"lastBuildDate":"Mon, 23 Oct 2023 13:40:15 +0900",
	"total":847155,
	"start":300,
	"display":100,
	"items":[
		{
			"title":"당신을 위한 좋은 <b>시 모음<\/b>... 여덟",
			"link":"https:\/\/blog.naver.com\/byuri4876\/220508648384",
			"description":"좋은 <b>시 모음<\/b>~ 여덟... &lt;Jessica Simpson - When You Told Me You Loved Me&gt; 지난밤 그대에게 보내려고 써 둔 엽서 아침에 다시 보니 부끄럽다는 생각이 들어 성냥불을 붙였다 끝까지 타지 않고 남은 글자들 외 로 움 【이외수... ",
			"bloggername":"벼리의 부동산 길라잡이2",
			"bloggerlink":"blog.naver.com\/byuri4876",
			"postdate":"20151014"
		},
		{
			"title":"7월의 <b>시 모음<\/b> [7월의 시] [여름 시 ] [아름다운 시]",
			"link":"https:\/\/blog.naver.com\/scan77\/222017216880",
			"description":"7월에 관계되는 <b>시 모음<\/b> 벌써 한 해의 절반이 지나가고 있습니다. 왕성한 생명력이 넘쳐나는 뜨거운... 6월을 보내고 새로운 7월을 맞이하면서 7월에 관계되는 시 몇 편을 올려봅니다. 7월의 시 이해인 \/ 수녀, 시인... ",
			"bloggername":"귀공자의 블로그",
			"bloggerlink":"blog.naver.com\/scan77",
			"postdate":"20200630"
		},
		{
			"title":"재미있는 <b>시 모음<